### Wprowadzenie - regresja liniowa
Oryginalny kod: https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn import linear_model
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

fig, axes = plt.subplots(1, 4, figsize=(15, 4))

for i, noise in enumerate([1.0, 10.0, 25.0, 50.0]):
    x, y, coef = make_regression(n_samples=100, n_features=1, noise=noise, random_state=0, coef=True)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0, shuffle=True)
    x = x.flatten()


    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(x_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(x_test)

    # The mean squared error
    print(f"RMSE: {mean_squared_error(y_test, y_pred)**.5:.2f}")

    # Plot outputs
    axes[i].scatter(x_test, y_test, color="black")    
    axes[i].plot([np.min(x_test),np.max(x_test)], [np.min(x_test)*coef,np.max(x_test)*coef], color="blue", linewidth=1, alpha=1.0)
    axes[i].plot(x_test, y_pred, color="red", linewidth=3, alpha=0.55)

    axes[i].set_xticks(())
    axes[i].set_yticks(())

    axes[i].set_xlabel("Dane wejściowe - X\n(Zmienna niezależna)")
    axes[0].set_ylabel("Dane wyjściowe - Y\n(Zmienna zależna)")


    axes[i].set_xlim(-2.5, 2.5)
    axes[i].set_ylim(-150, 150)

    axes[i].set_title(f"Noise={int(noise)}, R^2={r2_score(y_test, y_pred):.2f}, RMSE={mean_squared_error(y_test, y_pred)**.5:.1f}")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

DATASET_PATH = "Dane\\BostonHousing.csv" # Ścieżka do pliku z danymi

In [ ]:
# Wczytanie datasetu California Housing
dataset = pd.read_csv(DATASET_PATH)

In [ ]:
# Informacje o zestawie danych
dataset.info()

In [ ]:
# Własne etykiety i opisy dla cech
DATASET_FEATURE_DSCRB = {
    "crim": "Per capita crime rate by town",
    "zn": "Proportion of residential land zoned for lots over 25,000 sq.ft",
    "indus": "Proportion of non-retail business acres per town",
    "chas": "Charles River dummy variable (1 if tract bounds river; 0 otherwise)",
    "nox": "Nitric oxides concentration (parts per 10 million)",
    "RM": "Average number of rooms per dwelling",
    "age": "Proportion of owner-occupied units built prior to 1940",
    "dis": "Weighted distances to five Boston employment centres",
    "rad": "Index of accessibility to radial highways",
    "tax": "Full-value property-tax rate per $10,000",
    "ptratio": "Pupil-teacher ratio by town",
    "b": "1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town",
    "lstat": "[%] lower status of the population",
    "medv": "Median value of owner-occupied homes in $1000's",
}
DATASET_FEATURE_LABELS = {
    "crim": "crime",
    "zn": "zoned prop.",
    "indus": "industry prop.",
    "chas": "Charles River",
    "nox": "NOX",
    "rm": "No. rooms",
    "age": "pre-1940",
    "dis": "weighted dist.",
    "rad": "radial highways",
    "tax": "property tax",
    "ptratio": "pupil-teacher ratio",
    "b": "proportion of blacks",
    "lstat": "lower status",
    "medv": "median value",
}

In [ ]:
# Zmiana nazwy kolumn
dataset = dataset.rename(columns=DATASET_FEATURE_LABELS)
dataset.info()

### Brakujące dane

In [ ]:
# Znalezienie brakujących danych
dataset.isnull().any()

In [ ]:
# W rzędzie jest brakująca wartość
isnull = dataset.isnull().any(axis=1)
print(np.count_nonzero(isnull),"brakujących wartości.")
# Usunięcie rzędu
dataset = dataset.drop(np.asarray(isnull).nonzero()[0].tolist() ,axis=0)

In [ ]:
# Podgląd pierwszych 10 rzędów
dataset.head(10)

**Dane numeryczne**: 0, 1, -2, 1.37 ...

**Dane kategorialne**: "A", "samochód", "A23-1b"

In [ ]:
# Znalezienie kolumn z danymi kategorialnymi (nienumerycznymi)
dataset_cat=dataset.select_dtypes(include='object')
dataset_cat.columns

### Wydzielenie zmiennej zależnej

Zmienne wejściowe, cechy - zmienne **niezależne**

Zmienna wyjściowa, wynik - zmienna **zależna**

In [ ]:
# Wydzielenie zmiennej zależnej (Y)
x,y = dataset.drop(columns=["median value"]), dataset["median value"]

### EDA

Exploratory Data Analysis

Eksploracyjna Analiza Danych

In [ ]:
# Podstawowa analiza statystyczna
x.describe()

In [ ]:
# Histogramy zmiennych niezależnych
x.hist(figsize=(15,10), bins=20, edgecolor='black', grid=False)

In [ ]:
# Histogram zmiennej zależnej
fig,axes = plt.subplots(figsize=(5,3))
axes.hist(y*1000, bins=20, edgecolor='black')
axes.set_xlabel('Median value of owner-occupied homes in $')
axes.set_ylabel('Frequency')
axes.set_title('Histogram of Median Value')
plt.show()


In [ ]:
# Wykresy BOXPLOT

import seaborn as sns
import matplotlib.pyplot as plt

# Zmienna zależna Y(median value) w zależności od ilości pokojów 
plt.clf()
sns.boxplot(y=y, x=x["No. rooms"].round())
plt.show()

plt.clf()
sns.boxplot(y=x["No. rooms"], x=y.round(-1))
plt.show()

In [ ]:
# Analiza korelacji pomiędzy zmiennymi

import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(x.select_dtypes(exclude='object').corr(), ax=ax, annot=True, fmt=".2f", cmap="coolwarm", center=0)

### Train, Test, Val split

Podział na trzy **niezależne** zestawy:
* train - przygotowanie modelu
* val - ocena i strojenie modelu
* test - ostateczna ewaluacja

In [ ]:
# Podział na subsety TRAIN, TEST oraz VAL

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.4, random_state=42, shuffle=True)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42, shuffle=True)

In [ ]:
x_train.describe()

In [ ]:
x_val.describe()

In [ ]:
x_test.describe()

### Skalowanie wartości niezależnych

In [ ]:
# Skalowanie zmiennych niezależnych numerycznych z użyciem StandardScaler
# Enkodowanie zmiennych niezależnych kategorycznych z użyciem OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder 
from sklearn.compose import make_column_transformer


col_categorical = x_train.select_dtypes(include='object').columns # Kolumny kategoryczne
col_numerical = x_train.select_dtypes(exclude='object').columns # Kolumny numeryczne

# Transformer kolumn - pozwala skalować lub enkodować kolumny zależnie od ich rodzaju (numerical/categorical)
col_transformer = make_column_transformer(
    (StandardScaler(), col_numerical),
    (OrdinalEncoder(), col_categorical)
)

### Regresja - model regresji linowej, drzewo decyzyjne

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
# Regresja liniowa

from sklearn.linear_model import LinearRegression

# Pipeline dla regresji liniowej
linear_regressor = Pipeline([
    ('col_transformer', col_transformer),
    ('linear_regressor', LinearRegression(positive=True))
])
linear_regressor.fit(x_train, y_train)

y_pred = linear_regressor.predict(x_train)
rmse_linear = mean_squared_error(y_train, y_pred)**.5
r2_linear = r2_score(y_train, y_pred)
print(f"RMSE: {rmse_linear:.2f}")
print(f"R^2: {r2_linear:.3f}")

fig, ax = plt.subplots(1,1,figsize=(5,5))

ax.plot(y_train, y_pred, '.')
ax.plot([0, np.max(y_val)], [0, np.max(y_val)], color='red', linestyle='--', linewidth=1)
ax.set_xlabel("Oczekiwane wartości")
ax.set_ylabel("Predykcje")
ax.set_title("Zestaw treningowy")
ax.grid(True)
plt.show()


In [ ]:
# Drzewo decyzyjne

from sklearn.tree import DecisionTreeRegressor

# Pipeline dla drzewa decyzyjnego
decision_tree = Pipeline([
    ('col_transformer', col_transformer),
    ('decision_tree', DecisionTreeRegressor())
])
decision_tree.fit(x_train, y_train)

y_pred = decision_tree.predict(x_train)
rmse_tree = mean_squared_error(y_train, y_pred)**.5
r2_tree = r2_score(y_train, y_pred)
print(f"RMSE: {rmse_tree:.2f}")
print(f"R^2: {r2_tree:.3f}")

fig, ax = plt.subplots(1,1,figsize=(5,5))

ax.plot(y_train, y_pred, '.')
ax.plot([0, np.max(y_val)], [0, np.max(y_val)], color='red', linestyle='--', linewidth=1)
ax.set_xlabel("Oczekiwane wartości")
ax.set_ylabel("Predykcje")
ax.set_title("Zestaw treningowy")
ax.grid(True)
plt.show()

In [ ]:
print("Regresja liniowa vs Drzewo decyzyjne (test na zestawie treningowym)")
print(f"RMSE: {rmse_linear:.2f} vs {rmse_tree:.2f} (niższa wartość lepsza)")
print(f"R^2: {r2_linear:.3f} vs {r2_tree:.3f} (wyższa wartość lepsza)")

### Ewaluacja

In [ ]:
# Ewaluacja regresji liniowej na zestawie walidacyjnym

y_pred = linear_regressor.predict(x_val)
rmse_linear = mean_squared_error(y_val, y_pred)**.5
r2_linear = r2_score(y_val, y_pred)
print(f"RMSE: {rmse_linear:.2f}")
print(f"R^2: {r2_linear:.3f}")

fig, ax = plt.subplots(1,1,figsize=(5,5))

ax.plot(y_val, y_pred, '.')
ax.plot([0, np.max(y_val)], [0, np.max(y_val)], color='red', linestyle='--', linewidth=1)
ax.set_xlabel("Oczekiwane wartości")
ax.set_ylabel("Predykcje")
ax.set_title("Zestaw walidacyjny")
ax.grid(True)
plt.show()

In [ ]:
# Ewaluacja drzewa decyzyjnego na zestawie walidacyjnym

y_pred = decision_tree.predict(x_val)
rmse_tree = mean_squared_error(y_val, y_pred)**.5
r2_tree = r2_score(y_val, y_pred)
print(f"RMSE: {rmse_tree:.2f}")
print(f"R^2: {r2_tree:.3f}")

fig, ax = plt.subplots(1,1,figsize=(5,5))

ax.plot(y_val, y_pred, '.')
ax.plot([0, np.max(y_val)], [0, np.max(y_val)], color='red', linestyle='--', linewidth=1)
ax.set_xlabel("Oczekiwane wartości")
ax.set_ylabel("Predykcje")
ax.set_title("Zestaw walidacyjny")
ax.grid(True)
plt.show()

In [ ]:
print("Regresja liniowa vs Drzewo decyzyjne (test na zestawie walidacyjnym)")
print(f"RMSE: {rmse_linear:.2f} vs {rmse_tree:.2f} (niższa wartość lepsza)")
print(f"R^2: {r2_linear:.3f} vs {r2_tree:.3f} (wyższa wartość lepsza)")

### Strojenie hiperparametrów

In [ ]:
# Listowanie parametrów drzewa decyzyjnego
decision_tree['decision_tree'].get_params()

In [ ]:
# Dobór wielkości drzewa decyzyjnego
rmse = []
depths = [1, 2, 5, 10, 15, 20, 25, 50]
for depth in depths:

    decision_tree.set_params(**{'decision_tree__max_depth': depth})
    decision_tree.fit(x_val, y_val)
    y_pred = decision_tree.predict(x_val)
    rmse.append(mean_squared_error(y_val, y_pred)**.5)

best_depth, best_rmse = depths[np.argmin(rmse)], np.min(rmse)
    

print(f"Best RMSE: {best_rmse:.2f} dla max_depth={best_depth}")


In [ ]:
# Zastosowanie hiperparametrów i ponowny trening
decision_tree.set_params(**{'decision_tree__max_depth': best_depth})

decision_tree.fit(x_train, y_train)

In [ ]:
# Ewaluacja na zestawie walidacyjnym
y_pred = decision_tree.predict(x_val)
rmse_tree_tuned = mean_squared_error(y_val, y_pred)**.5
r2_tree_tuned = r2_score(y_val, y_pred)
print(f"RMSE: {mean_squared_error(y_val, y_pred)**.5:.2f}")
print(f"R^2: {r2_score(y_val, y_pred):.3f}")

fig, ax = plt.subplots(1,1,figsize=(5,5))

ax.plot(y_val, y_pred, '.')
ax.plot([0, np.max(y_val)], [0, np.max(y_val)], color='red', linestyle='--', linewidth=1)
ax.set_xlabel("Oczekiwane wartości")
ax.set_ylabel("Predykcje")
ax.set_title("Zestaw walidacyjny")
ax.grid(True)
plt.show()

In [ ]:
print("Drzewo decyzyjne vs Drzewo decyzyjne ze strojonymi hiperparametrami (test na zestawie walidacyjnym)")
print(f"RMSE: {rmse_tree:.2f} vs {rmse_tree_tuned:.2f} (niższa wartość lepsza)")
print(f"R^2: {r2_tree:.3f} vs {r2_tree_tuned:.3f} (wyższa wartość lepsza)")

### Selekcja cech metodą LASSO

In [ ]:
# Regresja Lasso do oceny ważności zmiennych niezależnych

from sklearn.linear_model import Lasso

lasso = Pipeline([
    ('col_transformer', col_transformer),
    ('lasso', Lasso(alpha=1e-05, max_iter=4000))
])

lasso.fit(x_train, y_train)

In [ ]:
# Wyznaczenie ważności zmiennych niezależnych i ich wizualizacja

lasso_coef = np.abs(lasso['lasso'].coef_)
lasso_coef /= np.sum(lasso_coef)

THRESH = 0.045

decision_tree_coef = np.abs(decision_tree['decision_tree'].feature_importances_)
decision_tree_coef /= np.sum(decision_tree_coef)

# plotting the Column Names and Importance of Columns. 
fig,axes = plt.subplots(1,2,figsize=(10,3))

axes[0].bar(x_train.columns.values, lasso_coef)
axes[0].axhline(y=THRESH, color='r', linestyle='-')
axes[0].grid()
axes[0].set_xticks(x_train.columns.values)
axes[0].set_xticklabels(x_train.columns.values, rotation = 90)
axes[0].set_title("Ważność cech wyznaczona metodą LASSO")
axes[0].set_xlabel("Nazwa cechy")
axes[0].set_ylabel("Wpływ")

axes[1].bar(x_train.columns.values, decision_tree_coef)
axes[1].grid()
axes[1].set_xticks(x_train.columns.values)
axes[1].set_xticklabels(x_train.columns.values, rotation = 90)
axes[1].set_title("Wpływ cech na model - Decision Tree")
axes[1].set_xlabel("Nazwa cechy")
axes[1].set_ylabel("Wpływ")

plt.show()

features_selected = x_train.columns[lasso_coef > THRESH]
features_ignored = x_train.columns[lasso_coef <= THRESH]
print(features_selected)

In [ ]:
# Usunięcie zmiennych nieistotnych
x_train = x_train.drop(columns=features_ignored)
x_val = x_val.drop(columns=features_ignored)
x_test = x_test.drop(columns=features_ignored)

In [ ]:
# Ponowne stworzenie pipeline'u oraz trenowanie modelu drzewa decyzyjnego dla zestawu z nowymi zmiennymi

col_categorical = x_train.select_dtypes(include='object').columns
col_numerical = x_train.select_dtypes(exclude='object').columns

col_transformer = make_column_transformer(
    (StandardScaler(), col_numerical),
    (OrdinalEncoder(), col_categorical)
)

decision_tree = Pipeline([
    ('col_transformer', col_transformer),
    ('decision_tree', DecisionTreeRegressor(max_depth=best_depth))
])


decision_tree.fit(x_train, y_train)

In [ ]:
# Wyznaczenie wpłwu cech niezależnych na model

decision_tree_coef = np.abs(decision_tree['decision_tree'].feature_importances_)
decision_tree_coef /= np.sum(decision_tree_coef)

# plotting the Column Names and Importance of Columns. 
fig,ax = plt.subplots(1,1,figsize=(5,3))

ax.bar(x_train.columns.values, decision_tree_coef)
ax.grid()
ax.set_xticks(x_train.columns.values)
ax.set_xticklabels(x_train.columns.values, rotation = 90)
ax.set_title("Wpływ cech na model - Decision Tree")
ax.set_xlabel("Nazwa cechy")
ax.set_ylabel("Wpływ")

plt.show()

In [ ]:
# Ewaluacja na zestawie walidacyjnym
y_pred = decision_tree.predict(x_val)
rmse_tree_fewerftrs = mean_squared_error(y_val, y_pred)**.5
r2_tree_fewerftrs = r2_score(y_val, y_pred)
print(f"RMSE: {rmse_tree_fewerftrs:.2f}")
print(f"R^2: {r2_tree_fewerftrs:.3f}")

fig, ax = plt.subplots(1,1,figsize=(5,5))

ax.plot(y_val, y_pred, '.')
ax.plot([0, np.max(y_val)], [0, np.max(y_val)], color='red', linestyle='--', linewidth=1)
ax.set_xlabel("Oczekiwane wartości")
ax.set_ylabel("Predykcje")
ax.set_title("Zestaw walidacyjny")
ax.grid(True)
plt.show()

In [ ]:
print("Drzewo decyzyjne vs Drzewo decyzyjne bez nieistotnych cech (test na zestawie walidacyjnym)")
print(f"RMSE: {rmse_tree:.2f} vs {rmse_tree_fewerftrs:.2f} (niższa wartość lepsza)")
print(f"R^2: {r2_tree:.3f} vs {r2_tree_fewerftrs:.3f} (wyższa wartość lepsza)")

### Walidacja na zbiorze testowym (finalna)

In [ ]:
y_pred = decision_tree.predict(x_test)
print(f"RMSE: {mean_squared_error(y_test, y_pred)**.5:.2f}")
print(f"R^2: {r2_score(y_test, y_pred):.3f}")

fig, ax = plt.subplots(1,1,figsize=(5,5))

ax.plot(y_test, y_pred, '.')
ax.plot([0, np.max(y_val)], [0, np.max(y_val)], color='red', linestyle='--', linewidth=1)
ax.set_xlabel("Oczekiwane wartości")
ax.set_ylabel("Predykcje")
ax.set_title("Zestaw testowy") 
ax.grid(True)
plt.show()

Jak zmiana *SEED'a* pdoczas podziału na zestawy train/test/val wpływa na wyniki modelu?